Set the working directory to the EMFieldML directory.

In [ ]:
import os
import sys
from pathlib import Path

# Add the project root to Python path
project_root = Path(__file__).parent.parent if '__file__' in globals() else Path.cwd().parent
sys.path.insert(0, str(project_root))

# Change to project root directory
os.chdir(project_root)

In [ ]:
# Clean imports from main package
from EMFieldML import (
    ActiveLearning,
    Decide6points,
    FekoRunner,
    FieldGPRegressor,
    TargetDataBuilder,
    PolyCubeMaker,
    Visualize,
    config,
    paths,
    template
)

# Import Utils classes directly (not available in main package due to circular dependencies)
from EMFieldML.Utils.DataSelect import DataSelect
from EMFieldML.Utils.ShieldModeler import ShieldModeler

In [ ]:
config.show_attrs()

In [ ]:
paths.show_attrs()

In [ ]:
template.show_attrs()

# Make shields
Generate parameters for all the shields and positions to be created, and based on these parameters, create the STL files for the shields.

### Make the shield parameter
Generate parameters for all the shields (372 types).

Additionally, data will be generated at 169 different positions for each shield.

Generated shield parameters will be saved in data/circle.

In [ ]:
ShieldModeler.make_shield_coordinate()

### Create the STL file for the shield
Create STL files for the shields using FEKO based on the generated shield parameters.

In an environment with FEKO installed, running this will allow you to execute a Lua file from a Python file and create an STL file in FEKO.

Generated STL files will be saved in data/stl.

In [ ]:
FekoRunner.make_stl_list()

The coil shape is fixed, but an STL file needs to be created for later visualization. As with the above, execute this in an environment with FEKO. 

The generated file will be saved in data/stl.

In [ ]:
FekoRunner.make_coil_stl()

# Make initial simulated data
Select the initial data for the training dataset, and perform simulations using FEKO on these data to obtain electromagnetic field data.

### Select initial 744 data
Select the initial data for training dataset.

The initial data were selected by choosing each shield twice at regular intervals.

In [ ]:
DataSelect.generate_x_train_init()

DataSelect.generate_x_test_init()

### Simulate initial data
Perform simulations using FEKO on these data to obtain electromagnetic field data.

Execute this in an environment with FEKO. 

In [ ]:
load_training_number_file_name = paths.TRAIN_DIR / template.x_train_init
x_train_list = []
with open(load_training_number_file_name) as f:
    for num in f:
        x_train_list.append(int(num))

FekoRunner.run_simulation_list(x_train_list)

# Make PolyCube mesh and Exterior Grid
Create PolyCube meshes and Exterior Grid.
Convert PolyCube meshes to x_data so that it can be used for training.
In addition, convert Exterior Grid to all position version.

In [ ]:
# It takes about 30 seconds to run this code as one shield shape.
# If you want to test this, I recommend to change n_shield_shape to 1.
# There is still a minor bug in the way points are taken, so any anomalies will need to be manually corrected, but most shield shapes work without issues.

PolyCubeMaker.make_modeling_all(n_shield_shape = 1)

From a single shield shape's PolyCube mesh, we create 169 x_data.

In [ ]:
PolyCubeMaker.move_shield_polycube_all(n_shield_shape = 1)

Output exterior grid for need positions in init learning.

In [ ]:
# Create the Exterior Grid when the coil is moved from the reference position's Exterior Grid.

PolyCubeMaker.move_shield_exterior_need()

# Active learning 
We perform active learning. In active learning, data is collected based on 6 prediction points. First, we select these six points, and then we perform active learning.

## Select 6 points
Select 6 points for active learning.

### Create patterns
Create patterns to classify points. This classification is based on the grouping of coordinates. Ensure that the six selected points are not chosen from the same pattern.

In [ ]:
Decide6points.make_pattern_list()

### Make y_data of initial data
Create y_data from the results of the first 744 simulation data points.

In [ ]:
train_list_path = paths.TRAIN_DIR / template.x_train_init

with open(train_list_path) as f:
    x_train_list = [int(s.rstrip()) for s in f.readlines()]

TargetDataBuilder.make_new_ydatafile(
    x_train_list,
    paths.Y_DATA_FOR_ACTIVE_LEARNING_DIR,
    prediction_point_list = list(range(config.n_prediction_points_level1)),
    n_prediction_points = config.n_prediction_points_level1,
    )

### Make learning model for each points
Create a learning model for each prediction point. 

The number of prediction points is 756 for grid level 1.

The models are created with the same parameters.

In [ ]:
Decide6points.make_learning_model_all()

### Make deviation result by predicting magnetic field at each prediction point
Calculate the deviation of the remaining unlabelled data using the trained model for all prediction points.

It takes about 5 min for each point. Separating the points and doing by two or more computers is recommendation.

In [ ]:
Decide6points.predict_deviation_all()

Check the deviation result and select the 6 prediction points for active learning.

In [ ]:
Decide6points.check_deviation()

# Selected point: 210  (Pattern: 12, Deviation: 2.886390965722237)
# Selected point: 540  (Pattern: 14, Deviation: 2.8818332883404496)
# Selected point: 276  (Pattern: 13, Deviation: 2.878365070348852)
# Selected point: 266  (Pattern: 10, Deviation: 2.8388720402344605)
# Selected point: 327  (Pattern: 19, Deviation: 2.7751982909967725)
# Selected point: 128  (Pattern: 3, Deviation: 2.6908574137764703)

## Active Learning
Perform Active Learning until the data reaches 1488 samples.
Execute this in an environment with FEKO. 

In [ ]:
ActiveLearning.uncertainly_sampling()

After active learning, `x_train.txt` and `x_test.txt` were saved as `x_train_after_active_learning.txt` and `x_test_after_active_learning.txt`, respectively.

# Learning
Train a model using data selected through Active Learning.

### Make y data
Create the y_data for the data obtained after Active Learning.

In [ ]:
train_list_path = paths.TRAIN_DIR / template.x_train_after_active_learning

with open(train_list_path) as f:
    x_train_list = [int(s.rstrip()) for s in f.readlines()]

TargetDataBuilder.make_new_ydatafile(x_train_list, paths.Y_DATA_DIR)

TargetDataBuilder.make_new_ydatafile_vector(x_train_list, paths.Y_DATA_DIR)

TargetDataBuilder.make_new_ydatafile_efficiency(x_train_list, paths.Y_DATA_DIR)

### Make model for efficiency
Create a model for efficiency.

In [ ]:
input_path_train_list = paths.TRAIN_DIR / template.x_train_after_active_learning
x_train_data = FieldGPRegressor.make_x_train_data(input_path_train_list)
input_path_y_data = paths.Y_DATA_DIR / template.y_data_efficiency
output_path = paths.LEARNING_MODEL_DIR / template.model
FieldGPRegressor.learning_magneticfield(input_path_y_data, output_path, x_train_data, lr_record = 0.015, iter_record = 2000, inter_record = 800, small_record = 5.0, initial_noise = 1e-4)

# Visualize
Visualize the shield and its magnetic field, and make it interactive.

In [ ]:
shield_number = 90
visualize = Visualize(shield_number)
visualize.visualize()